---
title: "2023_Kronensicherung_Plesse_Combination_and_Cleaning"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Combination_and_Cleaning

## Kombinieren und Bereinigen der Daten von LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
# Struktur
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
from IPython.display import Markdown, display
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype


## IMPORT: Daten Import

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [2]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    main_path,
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)

In [3]:
from utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

In [4]:
# Liste aller Dateinamen
filenames = [
    "tree.feather",
    "sensor.feather",
    "sensor_data_dict.json",
    "series.feather",
    "series_data_dict.json",
    "ls3_data_dict.json",
    "ls3.feather",
    "ptq_data_dict.json",
    "ptq.feather",
]

In [5]:
def load_file(path: Path):
    try:
        if path.suffix == ".feather":
            df_now = pd.read_feather(path)
            print(f"[Info] Feather geladen: {path.name}")
            return "df", path.stem, df_now
        elif path.suffix == ".json":
            with open(path, "r", encoding="utf-8") as f:
                data_dict_now = json.load(f)
                print(f"[Info] JSON geladen: {path.name}")
                return "dict", path.stem, data_dict_now
        else:
            print(f"[Warnung] Unbekannter Dateityp übersprungen: {path.name}")
            return None
    except Exception as e:
        print(f"[Fehler] Laden fehlgeschlagen für '{path.name}': {e}")
        return None


In [6]:
# Container
dfs = {}
data_dicts = {}

for file in filenames:
    result = load_file(data_export_directory / file)
    if result:
        kind, name_stem, content = result
        if kind == "df":
            dfs[f"{name_stem}_df"] = content
        elif kind == "dict":
            data_dicts[f"{name_stem}"] = content

[Info] Feather geladen: tree.feather
[Info] Feather geladen: sensor.feather
[Info] JSON geladen: sensor_data_dict.json
[Info] Feather geladen: series.feather
[Info] JSON geladen: series_data_dict.json
[Info] JSON geladen: ls3_data_dict.json
[Info] Feather geladen: ls3.feather
[Info] JSON geladen: ptq_data_dict.json
[Info] Feather geladen: ptq.feather


In [7]:
dfs.keys()

dict_keys(['tree_df', 'sensor_df', 'series_df', 'ls3_df', 'ptq_df'])

In [8]:
data_dicts.keys()

dict_keys(['sensor_data_dict', 'series_data_dict', 'ls3_data_dict', 'ptq_data_dict'])

In [9]:
# IDE-Hints: Feather-Dateien (DataFrames)
tree_df: pd.DataFrame = dfs.get("tree_df")
sensor_df: pd.DataFrame = dfs.get("sensor_df")
series_df: pd.DataFrame = dfs.get("series_df")
ls3_df: pd.DataFrame = dfs.get("ls3_df")
ptq_df: pd.DataFrame = dfs.get("ptq_df")

# IDE-Hints: Data Dictionary Dateien (JSON → dict)
sensor_data_dict: dict = data_dicts.get("sensor_data_dict")
series_data_dict: dict = data_dicts.get("series_data_dict")
ls3_data_dict: dict = data_dicts.get("ls3_data_dict")
ptq_data_dict: dict = data_dicts.get("ptq_data_dict")


## MERGE: Zusammenführen der Daten von LS3, PTQ und Versuchsprotokoll

Beginne mit PTQ-Daten, füge dann schrittweise die Daten des Versuchsablaufes, der Sensorpositionierung und Kraftmessung hinzu

In [10]:
df = pd.merge(ptq_df, series_df, on='id', how='left')
# df

Ergänze die Daten der Sensorpositionierung

In [11]:
# Merging df and elasto_df
# Geräte vom Typ Elastometer auswählen aus dem sensor_df
elasto_df = sensor_df[sensor_df["type"] == "Elasto"].copy()

# Passe die sensor_namen einheitlich für beide DataFrames an
elasto_df["sensor_name"] = elasto_df["type"].astype(str) + "(" + elasto_df["sensor_id"].astype(str) + ")"

# Perform the left join on sensor_name
df = df.merge(elasto_df, on="sensor_name", how="left")

# Gewünschte Reihenfolge
elasto_names = ["Elasto(90)", "Elasto(92)", "Elasto(95)", "Elasto(98)"]
# Definieren als geordneter CategoricalDtype
elasto_cat_type = CategoricalDtype(categories=elasto_names, ordered=True)
# Setzen des Typs auf die Spalte
df["sensor_name"] = df["sensor_name"].astype(elasto_cat_type)
# df

Ergänze die LS3-Daten

In [12]:
# Function to prepare ls3_metadata
def extract_ls3_subset(df, sensor_id, new_prefix):
    return (df.query(f"sensor_id == '{sensor_id}'")
            .drop('sensor_id', axis=1)
            .add_prefix(new_prefix)
            .rename(columns={f'{new_prefix}measurement_id': sensor_id}))

# Merging ls3_metadata with the main DataFrame
df = df.merge(extract_ls3_subset(ls3_df, '14:BF:E6', 'rope_'), on='14:BF:E6', how='left')     
df = df.merge(extract_ls3_subset(ls3_df, '14:99:1E', 'cable_'), on='14:99:1E', how='left')     

In [13]:
df.head(10)

,id,file_name,sensor_name,sample_rate,max_strain,max_compression,max_strain_osc,max_compression_osc,m_amplitude,m_amplitude_2,...,cable_timing_correction_factor,cable_datetime_start,cable_datetime_end,cable_duration,cable_length,cable_max,cable_mean,cable_median,cable_min,cable_release
0,1,PTQ_Meas_100346.txt,Elasto(90),4.003157,363.5,-168.1,363.5,-168.1,265.80,162.10,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,PTQ_Meas_100346.txt,Elasto(92),3.941160,239.9,-86.6,239.9,-86.6,163.25,76.00,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,PTQ_Meas_100346.txt,Elasto(95),4.003157,190.2,-119.1,190.2,-119.1,154.65,82.25,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,PTQ_Meas_100346.txt,Elasto(98),3.941160,141.6,-132.8,141.6,-132.8,137.20,92.50,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,PTQ_Meas_101814.txt,Elasto(90),3.447691,417.5,-202.2,417.5,-202.2,309.85,162.95,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,PTQ_Meas_101814.txt,Elasto(92),3.447691,271.6,-111.0,271.6,-111.0,191.30,79.55,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,PTQ_Meas_101814.txt,Elasto(95),3.447691,226.9,-90.4,226.9,-90.4,158.65,82.05,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2,PTQ_Meas_101814.txt,Elasto(98),3.447691,175.5,-93.8,175.5,-93.8,134.65,91.35,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,PTQ_Meas_102314.txt,Elasto(90),3.649011,424.6,-180.0,424.6,-180.0,302.30,178.35,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3,PTQ_Meas_102314.txt,Elasto(92),3.649011,289.0,-87.0,289.0,-87.0,188.00,88.40,...,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### EXPORT: Ungefilterte Daten exportieren für Externe (.feather, .csv)
 

In [14]:
df.to_feather(data_export_directory / "_dataset_full.feather")
df.to_csv(data_export_directory / "_dataset_full.csv", sep=";", index=True, encoding="utf-8")

## CLEANING: Daten bereinigen und Filtern

In [15]:
# Liste der Spalten, die beibehalten werden sollen
select_cols = ['id', 'rope_datetime', 'treatment', 'release_force_target', 
               'rope_release', 'cable_max',
               'sensor_name', 'location', 'direction', 'height', 'diameter',
               'max_strain', 'max_compression',
               'm_amplitude', 'm_amplitude_2', 
               'initial_amplitude', 'damping_coeff', 'frequency_damped', 'phase_angle', 'y_shift', 'x_shift', 'frequency_undamped', 'damping_ratio', 
               'metrics_warning', 'pearson_r', 'nrmse', 'nmae'
               ]

# DataFrame filtern
df = (
    df[select_cols]
    .query("release_force_target in [2.0, 2.4, 2.8] and treatment in ['free', 'gefa_dynamic', 'cobra_static']")
    .copy()
)

In [16]:
# Nicht verwendete Kategorien entfernen
df["treatment"] = df["treatment"].cat.remove_unused_categories()

In [17]:
# cable_max enthält noch ein np.ndarray mit index etc., nur Messwerte F benötigt
df['cable_max'] = df['cable_max'].apply(
    lambda x: float(x[1]) if isinstance(x, (tuple, list, np.ndarray)) and len(x) > 1 else x
)

In [18]:
df.head(10)

,id,rope_datetime,treatment,release_force_target,rope_release,cable_max,sensor_name,location,direction,height,...,frequency_damped,phase_angle,y_shift,x_shift,frequency_undamped,damping_ratio,metrics_warning,pearson_r,nrmse,nmae
4,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(90),StB,elongation,16.55,...,0.441518,0.200000,-5.287164,0.116641,0.442936,0.503994,False,0.943158,0.038866,0.018011
5,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(92),StB,elongation,11.6,...,0.432441,-0.200000,-14.233643,-0.009730,0.436601,0.873595,False,0.929554,0.034560,0.016183
6,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(95),StA,elongation,11.6,...,0.422702,-0.200000,-19.011253,-0.123043,0.426319,0.823806,False,0.908058,0.044873,0.024274
7,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(98),StA,elongation,16.85,...,0.443529,-0.065967,-1.099515,0.118608,0.445993,0.663224,False,0.980663,0.029621,0.021087
8,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(90),StB,elongation,16.55,...,0.440654,0.200000,7.775263,0.097887,0.441317,0.344954,False,0.957381,0.043035,0.018619
9,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(92),StB,elongation,11.6,...,0.439249,-0.200000,7.070001,0.049121,0.441269,0.603287,False,0.932950,0.043734,0.021138
10,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(95),StA,elongation,11.6,...,0.436023,-0.200000,-3.785755,-0.075741,0.438127,0.618021,False,0.938307,0.039020,0.022498
11,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(98),StA,elongation,16.85,...,0.441350,-0.200000,-17.747289,-0.203345,0.442888,0.525070,False,0.974139,0.028693,0.020715
12,4,2022-03-23 11:54:33,free,2.8,2.73950,NaN,Elasto(90),StB,elongation,16.55,...,0.439065,0.200000,-23.445975,0.120929,0.439764,0.354677,False,0.945320,0.049764,0.021509
13,4,2022-03-23 11:54:33,free,2.8,2.73950,NaN,Elasto(92),StB,elongation,11.6,...,0.436850,-0.200000,-19.454721,0.061912,0.438829,0.598701,False,0.922665,0.050822,0.020659


### EDIT: Datendokumentation an zusammengeführte und gefilterte Daten anpassen

Passe Datendokumentation den zusammengeführten und gefilterten Daten an, ergänze weitere später berechnete Features, erstellt vollstände aktuelle Datendokumentation

In [19]:
# Lade das Dictionary mit der Daten Dokumentation
with open(data_path / "calc_strain_data_dict.json", "r", encoding="utf-8") as f:
    calc_strain_data_dict = json.load(f)
    
select_cols += list(calc_strain_data_dict.keys())

In [20]:
# Rope-Keys erzeugen
ls3_rope_data_dict = {f"rope_{k}": v for k, v in ls3_data_dict.items()}
# Cable-Keys erzeugen
ls3_cable_data_dict = {    f"cable_{k}": v for k, v in ls3_data_dict.items()}

In [31]:
# Alles zusammenführen
full_data_dict = {
    **sensor_data_dict,
    **series_data_dict,
    **ls3_rope_data_dict,
    **ls3_cable_data_dict,
    **ptq_data_dict,
    **calc_strain_data_dict
}

In [32]:
# 1. sensor → Deutsch
if "sensor" in full_data_dict:
    full_data_dict["sensor"]["Deutsch"] = "Elastometer"

# 2. cable_max → Beschreibung, Deutsch, Zeichen
if "cable_max" in full_data_dict:
    full_data_dict["cable_max"]["Beschreibung"] = "Maximale gemessene Kraftspitze in der KS"
    full_data_dict["cable_max"]["Deutsch"] = "Kraftspitze KS"
    full_data_dict["cable_max"]["Zeichen"] = "$F_{\\mathrm{cable, max}}$"

In [33]:
data_dict = {key: full_data_dict[key] for key in select_cols if key in full_data_dict}
data_dict

{'id': {'Kategorie': 'ptq',
  'Zeichen': 'ID',
  'Deutsch': 'ID Messung',
  'Datentyp': 'int64',
  'Einheit': '-',
  'Beschreibung': 'Eindeutige ID der Messung'},
 'rope_datetime': {'Kategorie': 'ls3',
  'Zeichen': '$t$',
  'Deutsch': 'Zeitstempel',
  'Datentyp': 'object',
  'Einheit': '-',
  'Beschreibung': 'Startzeitpunkt der Messung laut Gerät'},
 'treatment': {'Kategorie': 'series',
  'Zeichen': 'treatment',
  'Deutsch': 'Behandlungsvariante',
  'Datentyp': 'category',
  'Einheit': '-',
  'Beschreibung': 'Art der KS: \\texttt{free}, \\texttt{gefa\\_dynamic}, \\texttt{cobra\\_static}'},
 'release_force_target': {'Kategorie': 'series',
  'Zeichen': '$F_{\\mathrm{release,target}}$',
  'Deutsch': 'Vorspannkraft Soll',
  'Datentyp': 'float64',
  'Einheit': 'kN',
  'Beschreibung': 'Geplante Vorspannkraft im Zugseil bei Release'},
 'rope_release': {'Kategorie': 'ls3',
  'Zeichen': '$F_{\\mathrm{release}}$',
  'Deutsch': 'Vorspannkraft Ist',
  'Datentyp': 'float64',
  'Einheit': 'kN',
  'B

In [24]:
df.head(100)

,id,rope_datetime,treatment,release_force_target,rope_release,cable_max,sensor_name,location,direction,height,...,frequency_damped,phase_angle,y_shift,x_shift,frequency_undamped,damping_ratio,metrics_warning,pearson_r,nrmse,nmae
4,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(90),StB,elongation,16.55,...,0.441518,0.200000,-5.287164,0.116641,0.442936,0.503994,False,0.943158,0.038866,0.018011
5,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(92),StB,elongation,11.6,...,0.432441,-0.200000,-14.233643,-0.009730,0.436601,0.873595,False,0.929554,0.034560,0.016183
6,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(95),StA,elongation,11.6,...,0.422702,-0.200000,-19.011253,-0.123043,0.426319,0.823806,False,0.908058,0.044873,0.024274
7,2,2022-03-23 11:24:23,free,2.8,2.72310,NaN,Elasto(98),StA,elongation,16.85,...,0.443529,-0.065967,-1.099515,0.118608,0.445993,0.663224,False,0.980663,0.029621,0.021087
8,3,2022-03-23 11:31:12,free,2.8,2.76205,NaN,Elasto(90),StB,elongation,16.55,...,0.440654,0.200000,7.775263,0.097887,0.441317,0.344954,False,0.957381,0.043035,0.018619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,25,2022-03-23 16:44:31,cobra_static,2.4,2.45465,2.80,Elasto(98),StA,elongation,16.85,...,0.526233,0.200000,0.004065,-0.123942,0.530764,0.826263,False,0.939117,0.040525,0.028335
100,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,Elasto(90),StB,elongation,16.55,...,0.543593,-0.200000,-5.142911,0.088797,0.546794,0.682883,False,0.890900,0.044374,0.022288
101,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,Elasto(92),StB,elongation,11.6,...,0.350000,-0.200000,-2.964532,-0.079053,0.384487,2.857143,False,0.866183,0.057465,0.033882
102,26,2022-03-23 16:56:04,cobra_static,2.0,1.99840,2.33,Elasto(95),StA,elongation,11.6,...,0.466347,-0.200000,4.559008,-0.153434,0.492757,2.144327,True,0.747046,0.070126,0.040196


In [34]:
# In DataFrame umwandeln
data_dict_df =  pd.DataFrame.from_dict(data_dict, orient="index")

# In Markdown umwandeln und anzeigen
md_text = data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|                         | Kategorie       | Zeichen                           | Deutsch                              | Datentyp   | Einheit   | Beschreibung                                                              |
|-------------------------|-----------------|-----------------------------------|--------------------------------------|------------|-----------|---------------------------------------------------------------------------|
| id                      | ptq             | ID                                | ID Messung                           | int64      | -         | Eindeutige ID der Messung                                                 |
| rope_datetime           | ls3             | $t$                               | Zeitstempel                          | object     | -         | Startzeitpunkt der Messung laut Gerät                                     |
| treatment               | series          | treatment                         | Behandlungsvariante                  | category   | -         | Art der KS: \texttt{free}, \texttt{gefa\_dynamic}, \texttt{cobra\_static} |
| release_force_target    | series          | $F_{\mathrm{release,target}}$     | Vorspannkraft Soll                   | float64    | kN        | Geplante Vorspannkraft im Zugseil bei Release                             |
| rope_release            | ls3             | $F_{\mathrm{release}}$            | Vorspannkraft Ist                    | float64    | kN        | Tatsächlich realisierte Vorspannkraft im Zugseil bei Release              |
| cable_max               | ls3             | $F_{\mathrm{cable, max}}$         | Kraftspitze KS                       | object     | kN        | Maximale gemessene Kraftspitze in der KS                                  |
| sensor_name             | ptq             | sensor                            | Sensorname                           | object     | -         | Bezeichnung des Elastometers                                              |
| location                | sensor_position | location                          | Position                             | string     | -         | Position des Sensors am Stamm                                             |
| direction               | sensor_position | direction                         | Richtung                             | string     | -         | Zug- oder Druckseite                                                      |
| height                  | sensor_position | $h$                               | Höhe                                 | Float64    | m         | Höhe des Sensors am Stamm                                                 |
| diameter                | sensor_position | $d$                               | Durchmesser                          | Float64    | cm        | Durchmesser des Stammes                                                   |
| max_strain              | ptq             | $\Delta l_{\mathrm{max}}$         | Dehnung max. gemessen                | float64    | $\mu$m    | Gemessene maximale absolute Randfaserdehnung                              |
| max_compression         | ptq             | $\Delta l_{\mathrm{comp,max}}$    | Stauchung max. gemessen              | float64    | $\mu$m    | Gemessene maximale absolute Randfaserstauchung                            |
| m_amplitude             | ptq_osc         | $mA$                              | Manuelle Amplitude                   | float64    | $\mu$m    | Manuell berechnete Amplitude über den Schwingungsabschnitt                |
| m_amplitude_2           | ptq_osc         | $mA_2$                            | Manuelle Amplitude 2                 | float64    | $\mu$m    | Manuell berechnete Amplitude zwischen 2. Peak und Minimum                 |
| initial_amplitude       | ptq_osc         | $A$                               | Anfangsamplitude                     | float64    | $\mu$m    | Initiale Amplitude der angepassten Schwingung                             |
| damping_coeff           | ptq_osc         | $\delta$                          | Dämpfungskoeffizient                 | float64    | 1/s       | Koeffizient der exponentiellen Dämpfung                                   |
| frequency_damped        | ptq_osc         | $f_{\mathrm{d}}$                  | Gedämpfte Frequenz                   | float64    | Hz        | Frequenz der gedämpften Schwingung                                        |
| phase_angle             | ptq_osc         | $\varphi$                         | Phasenwinkel                         | float64    | rad       | Anfangsphase der Schwingung                                               |
| y_shift                 | ptq_osc         | $y_0$                             | Vertikaler Versatz                   | float64    | $\mu$m    | Vertikaler Offset der Schwingung                                          |
| x_shift                 | ptq_osc         | $t_0$                             | Zeitverschiebung                     | float64    | s         | Horizontale Verschiebung der Schwingung                                   |
| frequency_undamped      | ptq_osc         | $f_0$                             | Ungedämpfte Frequenz                 | float64    | Hz        | Frequenz der ungedämpften Schwingung                                      |
| damping_ratio           | ptq_osc         | $D$                               | Dämpfungsgrad                        | float64    | -         | Verhältnis von Dämpfung zu Frequenz                                       |
| metrics_warning         | ptq_osc         | warning                           | Fit-Warnung                          | bool       | -         | Warnung, wenn Qualitätsmetrik Schwellenwerte unterschreitet               |
| pearson_r               | ptq_osc_metric  | $r$                               | Pearson-Korrelation                  | float64    | -         | Korrelationskoeffizient der Anpassung                                     |
| nrmse                   | ptq_osc_metric  | $\mathrm{NRMSE}$                  | Normalisierter RMSE                  | float64    | -         | Normalisierter mittlerer quadratischer Fehler                             |
| nmae                    | ptq_osc_metric  | $\mathrm{NMAE}$                   | Normalisierter MAE                   | float64    | -         | Normalisierter mittlerer absoluter Fehler                                 |
| calc_max_strain         | calc_strain     | $\varepsilon_{\mathrm{calc,max}}$ | Dehnung max. berechnet               | float64    | $\mu$m    | Berechnete maximale Randfaserdehnung                                      |
| calc_max_strain_relativ | calc_strain     | $\varepsilon_{\mathrm{calc,rel}}$ | Berechnete maximale Randfaserdehnung | float64    | $\mu$m    | Differenz gemessene und berechnete maximale Randfaserdehnung              |
| strain_difference       | calc_strain     | $\Delta \varepsilon$              | Differenz der Faserdehnung           | float64    | \%        | Relative Differenz der berechneten und gemessenen max. Dehnung            |

### EXPORT: Gefilterte Daten und Datendokumentation exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [26]:
df.to_feather(data_export_directory / "_dataset_clean.feather")
df.to_csv(data_export_directory / "_dataset_clean.csv", sep=";", index=True, encoding="utf-8")

with open(data_export_directory / "_data_dict_clean.csv", "w", encoding="utf-8") as f:
    json.dump(data_dict, f, indent=4, ensure_ascii=False)

### LATEX-EXPORT: Datendokumentation als Latex-Tabelle exportieren (.tex)

In [27]:
# Erzeuge DataFrame für LaTeX mit einheitlichem Aufbau
data_dict_df = build_data_dict_df(data_dict)

# Exportiere als LaTeX
latex_string = data_dict_df.to_latex(index=False, escape=False)

caption = "Feldversuch 2 - Ergebnisse, Daten Dokumentation"
caption_long = "Feldversuch 2 - Ergebnisse, Daten Dokumentation, Kräfte, Dehnungen und Schwingungsparameter"

save_latex_table(latex_string, caption, latex_export_directory, caption_long=caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_daten_dokumentation.tex
